In [1]:
import joblib
import sys,os,time,platform


if platform.node() == 'aizat-fyp':
    datasetpath = '/home/aizat/OneDrive/Master Project/Workspace/dataset/ember2018'
    boostingpath ='/home/aizat/OneDrive/Master Project/Workspace/boosting-ember'
    sys.path.append(r'/home/aizat/ember')
    sys.path.append(r'/home/aizat/OneDrive/Master Project/Workspace/boosting-ember')
else:
    datasetpath = '/rapids/aizat/OneDrive/Master Project/Workspace/dataset/ember2018'
    boostingpath ='/rapids/aizat/OneDrive/Master Project/Workspace/boosting-ember'
    sys.path.append(r'/rapids/aizat/ember')
    sys.path.append(r'/rapids/aizat/OneDrive/Master Project/Workspace/boosting-ember')
'''
    datasetpath = '/root/dataset/ember2018/'
    boostingpath = '/root/boosting-ember'
    sys.path.append(r'/root/ember')
    sys.path.append(r'/root/boosting-ember')
'''
import mlflow    
import ember
import pandas as pd
import numpy as np
import lightgbm as lgb
import boostember
from boostember.features_extended import *
from GPUtil import showUtilization as gpusage

gpusage()
boostember.mlflowsetup(os.path.join(boostingpath, 'mlflow'))

| ID | GPU | MEM |
------------------
|  0 |  1% |  4% |


True

In [2]:
X_train, y_train, X_test, y_test = ember.read_vectorized_features(datasetpath)
delunlabel = (y_train != -1)

X_train = X_train[delunlabel]
#X_train = pd.DataFrame(X_train, columns=emberfeaturesheader())
y_train = y_train[delunlabel]
X_train.shape, y_train.shape

((600000, 2381), (600000,))

In [3]:
mlflow.set_tracking_uri("https://atlascompanion.live/")
#mlflow.create_experiment("", artifact_location="gs://mlflow_aizat")
mlflow.set_experiment("Demo")

In [4]:
def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

In [5]:
from mlflow.tracking import MlflowClient
import mlflow.pyfunc

client = MlflowClient()
client.list_artifacts("e82bce1a1b6542b88c982ef9302e7daf")
client.download_artifacts("e82bce1a1b6542b88c982ef9302e7daf", "best_estimator/model.pkl", "model")

model = joblib.load('./model/best_estimator/model.pkl')
#model = mlflow.pyfunc.load_model(model_uri=f"models:/Demo/1")


In [6]:
'''
import shap

shg_explainer = shap.explainers.GPUTree(model=model)
shg_shap_values = shg_explainer.shap_values(pd.DataFrame(X_test))
'''

'\nimport shap\n\nshg_explainer = shap.explainers.GPUTree(model=model)\nshg_shap_values = shg_explainer.shap_values(pd.DataFrame(X_test))\n'

In [7]:
from memory_profiler import profile
fp=open('memory_profiler.log','w+')

@profile(stream=fp)
def profilermem(fcn):
    return fcn

In [9]:
import shap


mlflow.autolog()

from pympler import tracker
tr = tracker.SummaryTracker()

with mlflow.start_run() as run:
    mlflow.set_tags({"description": "Demo",
    "model": "shap",
    "summary": "shap tree test"})
    sh_explainer = profilermem(shap.TreeExplainer(model=model))
    tr.print_diff()
    sh_shap_values = sh_explainer.shap_values(pd.DataFrame(X_test))
    


ERROR: Could not find file <ipython-input-7-aeb070167c32>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.
                             types |   # objects |   total size
================================== | =========== | ============
                     numpy.ndarray |          10 |    312.47 MB
                              list |       30014 |      2.53 MB
                               str |       32365 |      2.48 MB
                              dict |        1537 |    331.45 KB
                               int |        6623 |    181.75 KB
                              code |         953 |    164.97 KB
                              type |         133 |    118.04 KB
  shap.explainers._tree.SingleTree |        1000 |     46.88 KB
                             tuple |         520 |     27.33 KB
                           weakref |         167 |     11.74 KB
                               set |          34 |     11.67 KB
    

LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [ ]:
'''
from cuml.explainer import KernelExplainer

cu_explainer = KernelExplainer(model=model.predict, data=X_train, nsamples=2048)
cu_shap_values = cu_explainer.shap_values(X_test[:100])
''''''

In [ ]:
mlflow.log_artifact("features.txt", artifact_path="features")

In [ ]:
!pip install google-cloud-storage mlflow

In [ ]:
!pip install mlflow

In [10]:
fp.close()

In [11]:
mlflow.log_artifact('memory_profiler.log', 'memlog')


In [16]:
joblib.dump(sh_explainer, 'explainer.pkl')
mlflow.log_artifact('explainer.pkl', 'shap')

In [18]:
joblib.dump(sh_shap_values, 'sh_val.pkl')
mlflow.log_artifact('sh_val.pkl', 'shap')

KeyboardInterrupt: 